In [87]:
import os, sys, pandas as pd, glob
import datetime, time
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('mode.chained_assignment', None)

In [37]:
input_dir = "../scripts/data/*.csv"
input_data_file_list = glob.glob(input_dir)

li = []
for filename in input_data_file_list:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)
# filter out the lines from the 
df = df[~pd.isna(df["pr"])]


In [54]:
print(df)

                        ts                   te       td                   sa  \
0      2018-10-26 06:00:25  2018-10-26 06:04:35  250.011          192.168.1.1   
1      2018-10-26 06:04:36  2018-10-26 06:04:36    0.097        192.168.1.119   
2      2018-10-26 05:55:49  2018-10-26 06:00:20  270.818       74.125.200.188   
3      2018-10-26 05:59:26  2018-10-26 05:59:29    2.859       174.129.217.97   
4      2018-10-26 06:02:20  2018-10-26 06:02:20    0.000        192.168.1.108   
...                    ...                  ...      ...                  ...   
99311  2018-10-26 06:28:34  2018-10-26 06:28:41    6.694       174.129.217.97   
99312  2018-10-26 06:26:37  2018-10-26 06:26:37    0.000        192.168.1.119   
99313  2018-10-26 06:26:41  2018-10-26 06:26:41    0.042  fdc1:1bdc:1e84::911   
99314  2018-10-26 06:26:37  2018-10-26 06:26:37    0.008        192.168.1.129   
99315  2018-10-26 06:23:56  2018-10-26 06:28:26  269.997        192.168.1.118   

                      da   

In [64]:
def getTimestamp(time_str):
    return time.mktime(datetime.datetime.strptime(time_str, "%Y-%m-%d %H:%M:%S").timetuple())
    
df['st_timestamp'] = df['ts'].apply(getTimestamp)
df['et_timestamp'] = df['te'].apply(getTimestamp)

       st_timestamp                   ts
0     1540551625.00  2018-10-26 06:00:25
1     1540551876.00  2018-10-26 06:04:36
2     1540551349.00  2018-10-26 05:55:49
3     1540551566.00  2018-10-26 05:59:26
4     1540551740.00  2018-10-26 06:02:20
...             ...                  ...
99311 1540553314.00  2018-10-26 06:28:34
99312 1540553197.00  2018-10-26 06:26:37
99313 1540553201.00  2018-10-26 06:26:41
99314 1540553197.00  2018-10-26 06:26:37
99315 1540553036.00  2018-10-26 06:23:56

[98455 rows x 2 columns]


In [77]:
df_mac2ip = pd.read_csv('data/iot_mac2ip.csv', sep='\t')
df_mac2ip['fname'] = df_mac2ip['MAC'].apply(lambda x: x.replace(':', '') + '.csv')
mac2ip_list = df_mac2ip.values.tolist()
dfTemp = df
dfTemp['flowTag'] = ""

[['ec:1a:59:83:28:11', '192.168.1.165', 'WEMO Motion Sensor', 'ec1a59832811.csv'], ['ec:1a:59:79:f4:89', '192.168.1.223', 'WEMO Power Switch', 'ec1a5979f489.csv'], ['00:16:6c:ab:6b:88', '192.168.1.248', 'Samsung Camera', '00166cab6b88.csv'], ['50:c7:bf:00:56:39', '192.168.1.227', 'TP Link Plug', '50c7bf005639.csv'], ['70:ee:50:18:34:43', '192.168.1.241', 'Netatmo Camera', '70ee50183443.csv'], ['00:17:88:2B:9A:25', '192.168.1.129', 'Huebulb', '0017882B9A25.csv'], ['44:65:0D:56:CC:D3', '192.168.1.239', 'AmazonEcho', '44650D56CCD3.csv'], ['F4:F5:D8:8F:0A:3C', '192.168.1.119', 'chromecast', 'F4F5D88F0A3C.csv'], ['74:C6:3B:29:D7:1D', '192.168.1.163', 'ihome', '74C63B29D71D.csv'], ['D0:73:D5:01:83:08', '192.168.1.118', 'lifx', 'D073D5018308.csv']]


In [88]:
for i in range(0, len(mac2ip_list)):
    fname = '../data/annotations/' + mac2ip_list[i][3]
    ip_addr = mac2ip_list[i][1]
    print(fname)
    df_annotation = pd.read_csv(fname)
    l = df_annotation.values.tolist()
    for i in range(0, len(l)):
        v = l[i]
        (st, et) = (v[0], v[1])
        tag = v[2]
        dfTemp['flowTag'].loc[(df['st_timestamp'] >= st) & (df['st_timestamp'] < et) & ((df['sa'] == ip_addr) | (df['da'] == ip_addr)) ] = tag

    


../data/annotations/ec1a59832811.csv
../data/annotations/ec1a5979f489.csv
../data/annotations/00166cab6b88.csv
../data/annotations/50c7bf005639.csv
../data/annotations/70ee50183443.csv
../data/annotations/0017882B9A25.csv
../data/annotations/44650D56CCD3.csv
../data/annotations/F4F5D88F0A3C.csv
../data/annotations/74C63B29D71D.csv
../data/annotations/D073D5018308.csv


In [89]:
print(dfTemp[dfTemp['flowTag'] != '']['flowTag'])

Series([], Name: flowTag, dtype: object)


/usr/local/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setit

In [73]:
dfTemp[dfTemp['flowTag'] != '']['flowTag']

Series([], Name: flowTag, dtype: object)

In [20]:
ips_to_remap_list = ["192.168.1.1", "192.168.1.2"]
ips_to_remap_dict = dict.fromkeys(ips_to_remap_list, 1)

In [38]:
## try with the brute-force attack

                        ts                   te       td                   sa  \
0      2018-10-26 06:00:25  2018-10-26 06:04:35  250.011          192.168.1.1   
1      2018-10-26 06:04:36  2018-10-26 06:04:36    0.097        192.168.1.119   
2      2018-10-26 05:55:49  2018-10-26 06:00:20  270.818       74.125.200.188   
3      2018-10-26 05:59:26  2018-10-26 05:59:29    2.859       174.129.217.97   
4      2018-10-26 06:02:20  2018-10-26 06:02:20    0.000        192.168.1.108   
...                    ...                  ...      ...                  ...   
99311  2018-10-26 06:28:34  2018-10-26 06:28:41    6.694       174.129.217.97   
99312  2018-10-26 06:26:37  2018-10-26 06:26:37    0.000        192.168.1.119   
99313  2018-10-26 06:26:41  2018-10-26 06:26:41    0.042  fdc1:1bdc:1e84::911   
99314  2018-10-26 06:26:37  2018-10-26 06:26:37    0.008        192.168.1.129   
99315  2018-10-26 06:23:56  2018-10-26 06:28:26  269.997        192.168.1.118   

                      da   

In [41]:
#gets a list of IP addresses to remap 
def get_ip_pool(fname):
    df = pd.read_csv(fname, header=0)
    ip_list = list(df["ip"])
    return ip_list

ip_list = get_ip_pool("data/ip_list.csv")
last_remap_idx = 0
len_ip_list = len(ip_order_list)
flow_map = {}

# remap the IP address. Don't parallelize the function
def remap_ip(x, field_name):
    src_ip, src_port, dst_ip, dst_port = (x["sa"], x["sp"], x["da"], x["dp"])
    ip_to_map = src_ip
    if field_name == "da":
        ip_to_map = dst_ip
    
    global last_remap_idx, flow_map
    flow_tuple = (src_ip, src_port, dst_ip, dst_port)
    if ip_to_map in ips_to_remap_dict:
        if flow_tuple in flow_map:
            ip_to_map = flow_map[flow_tuple]
        else:
            ip_to_map = ip_list[last_remap_idx % len_ip_list]
            flow_map[flow_tuple] = ip_to_map
            last_remap_idx += 1
    return ip_to_map

In [42]:
df["src_ip_remapped"] = df.apply(lambda x: remap_ip(x, 'sa'), axis=1)
df["dst_ip_remapped"] = df.apply(lambda x: remap_ip(x, 'da'), axis=1)
outdir = "data/remapped_data/"
df.to_csv("data/remapped_data/")

[['2018-10-26 06:00:25', '2018-10-26 06:04:35', '250.011', '192.168.1.1', '192.168.1.119', '0', 0.0, 'ICMP', '........', 0.0, 0.0, 10.0, 560.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, '0.0.0.0', '0.0.0.0', 0.0, 0.0, '00:00:00:00:00:00', '00:00:00:00:00:00', '00:00:00:00:00:00', '00:00:00:00:00:00', '0-0-0', '0-0-0', '0-0-0', '0-0-0', '0-0-0', '0-0-0', '0-0-0', '0-0-0', '0-0-0', '0-0-0', 0.0, 0.0, 0.0, '0.0.0.0', '0/0', 0.0, '1969-12-31 18:00:00.000', '192.168.10.10', '192.168.1.119'], ['2018-10-26 06:04:36', '2018-10-26 06:04:36', '0.097', '192.168.1.119', '8.8.8.8', '36258', 53.0, 'UDP', '........', 0.0, 0.0, 1.0, 32.0, 1.0, 48.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, '0.0.0.0', '0.0.0.0', 0.0, 0.0, '00:00:00:00:00:00', '00:00:00:00:00:00', '00:00:00:00:00:00', '00:00:00:00:00:00', '0-0-0', '0-0-0', '0-0-0', '0-0-0', '0-0-0', '0-0-0', '0-0-0', '0-0-0', '0-0-0', '0-0-0', 0.0, 0.0, 0.0, '0.0.0.0', '0/0', 0.0, '1969-12-31 18:00:00.000', '192.168.1.119', '8.8.8.8'], ['2018-10-